## Business Understanding
In the rapidly evolving food industry, the ability to accurately classify food items through images is becoming increasingly critical. As consumers demand more personalized and efficient services, businesses are seeking innovative solutions to enhance their operational efficiency and customer satisfaction. The integration of AI-powered food image classification can revolutionize various aspects of the food ecosystem and  restaurant operations . By automating food identification, businesses can streamline processes, reduce human error, and provide a more engaging user experience.


## Problem statement
Accurate food classification remains a challenge in the food industry, affecting restaurants, delivery platforms, and nutrition tracking apps. Manual identification leads to errors in  menu categorization and automated checkouts.

`An AI-powered food classification model can automate this process with high accuracy, reducing errors, improving user experience, and enhancing operational efficiency.`

## Project Goals
### Optimization Strategies
Improving accuracy beyond 78% is crucial for reliable food identification. Advanced deep learning techniques, data augmentation, and model optimization can enhance classification performance, reducing errors in real-world applications.

### Impact on Automation and Service Efficiency
AI-powered food classification can streamline restaurant operations, minimize human error, and improve service efficiency. In food delivery, it optimizes workflow, enhances order accuracy, and reduces delays.


### Scalability and Real-World Application
For widespread adoption, the model must handle diverse food categories efficiently. Ensuring accuracy in automated checkouts, menu categorization, and large-scale applications is key to real-world feasibility.



## Stakeholders
##### Restaurants and Food Service Providers
They will benefit from improved operational efficiency, reduced errors in order processing, and enhanced customer satisfaction.
Food Delivery Platforms

-These platforms can leverage accurate food classification to streamline their logistics, improve menu categorization, and enhance user experience.

##### Consumers
End-users will gain from more accurate  personalized dietary recommendations, and a more seamless food ordering experience.

##### Health and Nutrition Apps
These applications can utilize the classification model to provide users with better insights into their dietary habits and nutritional intake.

##### Data Scientists and AI Developers
Professionals in this field will be engaged in developing and refining the classification model, contributing to advancements in AI technology.

## Beneficiaries
##### Consumers
They will experience improved accuracy in food selection

##### Restaurants
Enhanced operational efficiency and reduced errors will lead to cost savings and improved customer loyalty.

##### Food Delivery Services
Improved accuracy in food classification will streamline operations, reduce delivery times, and enhance customer satisfaction.

##### Health Professionals
They can utilize accurate food classification data to provide better dietary advice and support to their clients.

##### Technology Providers
Companies developing AI solutions will benefit from the demand for advanced food classification technologies, leading to potential partnerships and revenue growth.

`By addressing these business questions and engaging the identified stakeholders and beneficiaries, the project can create a significant impact on the food industry, enhancing both operational efficiency and consumer experience.`

## Data Understanding
The Food-101 dataset is a large-scale image dataset containing 101,000 images spanning 101 food categories, with 1,000 images per class. It was introduced in the paper "Food-101 – Mining Discriminative Components with Random Forests" by Lukas Bossard, Matthieu Guillaumin, and Luc Van Gool.

In [ ]:
# Import modules
import tensorflow as tf
import tensorflow_datasets as tfds


##### Dataset Structure
*Training Set*: 75,750 images (750 per class)

*Test Set*: 25,250 images (250 per class)

*Image Format*: RGB, 512 × 512 pixels

*Classes*: 101 different food items, including dishes like pizza, sushi, steak, and ramen



##### Data Characteristics
*Imbalance*: The dataset is evenly distributed across all 101 food categories.

*Quality Issues*: The training set contains some noisy labels, making it slightly challenging for model training.

*Data Augmentation*: Since the dataset lacks variations in angles, lighting, and occlusions, augmentation techniques like 
rotation, flipping, and color jittering can improve model generalization.

##### Data-Source-tensorflowdatasets(tfds)
tfds- is an online source[https://www.tensorflow.org/datasets/catalog/food101?hl=en]

In [ ]:
# List all available datasets and check if the Food101 dataset is present in the tensorflow dataset
dataset_list =tfds.list_builders()
print('food101' in dataset_list)


In [ ]:
# Load in the data () Takes a while atleast 10 minutes
(train_data,test_data), ds_info =tfds.load(name = 'food101',
                                           split = ['train','validation'],
                                           shuffle_files=True,
                                           as_supervised=True, # data gets returned in tuple format (data,label)
                                           with_info=True)

##### Importance of the Dataset
The Food-101 dataset is a widely used benchmark for food classification, offering 101,000 images across 101 diverse food categories. It is valuable for applications like restaurant recommendation systems, calorie estimation, and AI-driven dietary monitoring.

The dataset supports ;

*Fine-Grained Classification: Allows for detailed and accurate classification of similar food items.*

*State-of-the-Art Models: Leverages advanced models for robust and efficient performance.*

*Global Variety: Ensures models are generalizable across diverse culinary contexts.*

*Real-World Noisy Labels: Prepares models to handle imperfections and real-world conditions.*

## Inspect the Food 101 dataset

### By becoming one with the data we aim to get:
* `class names` - we're working with 101 different food classes
* The shape of our input data (image tensors)
* The datatype of our input data
* What the labels look like (e.g are they one-hot encoded or are they label encoded)
* Do the labels match up with the class names?

In [ ]:
#  Get the class names
class_names=ds_info.features['label'].names
print('Length:',len(class_names))
class_names[:10] # Extract the first 10 names

In [ ]:
# Take on sample of the train data
train_one_sample = train_data.take(1)  #(image_tensor,label)

In [ ]:
train_one_sample

In [ ]:
for image,labels in train_one_sample:
  print(f"""
  Image shape: {image.shape}
  Image datatype: {image.dtype}
  Target class from Food101 (tensor form): {labels}
  Class names (str form): {class_names[labels.numpy()]}
  """)

In [ ]:
# How the image tensors look like
image

In [ ]:
# what are the min and max values of image tensor?
tf.reduce_min(image),tf.reduce_max(image)

In [ ]:
# PLot an image tensor
import matplotlib.pyplot as plt

plt.figure(figsize=(10,7))
plt.imshow(image)
plt.title(class_names[labels.numpy()])
plt.axis('off');
image.shape

In [ ]:
## Create a function that plots a given number of  random image from the TFDS Food101 dataset
def TFDS_plot(train_data,nrows=2,ncol=5,Class_names =class_names,plot_no =10):
  #Loop through the sample and extract the label and image

# Plot the data
   images = []
   labels=[]

   for image,label in train_data.take(plot_no):
      images.append(image),
      labels.append(label)

   plt.figure(figsize=(10,8))
   for i in range(plot_no):
      k = i+ 1
      plot_data=plt.subplot(nrows,ncol,k) # has to be adjusted based
      plot_data=plt.imshow(images[i])
      plot_data=plt.title(Class_names[labels[i].numpy()])
      plot_data=plt.axis('off')
      plot_data=plt.tight_layout()
      i += 1
   return plot_data

In [ ]:
TFDS_plot(train_data)

## Data preprocessing

### Creating Preprocessing Functions for Our Data  

Neural networks achieve optimal performance when data is formatted in a specific way (e.g., batched, normalized, etc.). However, raw data—especially from TensorFlow datasets—often requires preprocessing to meet these requirements.  

#### Key Characteristics of Our Data:  
- Stored in `uint8` format  
- Contains images of varying sizes  
- Pixel values range from 0 to 255 (not yet normalized)  

#### What Our Model Prefers:  
- Data in `float32` format (or `float16`/`float32` for mixed precision)  
- Uniform image sizes within each batch  
- Scaled pixel values (0 to 1) for improved model performance  

#### Preprocessing Requirements:  
Since we are using an **EfficientNetBX** pretrained model from `tf.keras.applications`, explicit rescaling is unnecessary as these models include built-in rescaling.  

Thus, our preprocessing function should:  
1. Resize all images to a consistent shape.  
2. Convert image tensors from `uint8` to `float32`.

In [ ]:
# Make a function for preprocessing images
def preprocess_img(image,label,img_shape=224):
  """
  Converts image datatype from `uint8` -> `float 32` and reshapes
  the image shape and color channels-|
  [img_shape,img_shape,color channel]
  """
  image =tf.image.resize(image,[img_shape,img_shape]) # Reshape target image
  # image =image/255. # scale image value (Depends on the model in use)
  return tf.cast(image,tf.float32), label #return (float32_image, label) tuple


### Batch & Prepare datasets

We're going to make our data input pipeline run really fast.

For more resources on this, I'd highlighly recommend [Pipipeline Introduction:](https://www.tensorflow.org/guide/data)



In [ ]:
# Map preprocessing functions to training (and parallelize)
train_data =train_data.map(map_func=preprocess_img,num_parallel_calls=tf.data.AUTOTUNE)

# Shuffle train_data and turn it into batches and prefech it (load it faster)
train_data = train_data.shuffle(buffer_size=1000).batch(batch_size =32).prefetch(buffer_size = tf.data.AUTOTUNE)

# Map preprocessing function to test data
test_data =test_data.map(map_func=preprocess_img,num_parallel_calls=tf.data.AUTOTUNE)
test_data =test_data.batch(32).prefetch(buffer_size =tf.data.AUTOTUNE) # No need to shuffle


In [ ]:
train_data,test_data

### Setup mixed precision training

Mixed precision is the use of both 16-bit and 32-bit floating-point types in a model during training to make it run faster and use less memory. By keeping certain parts of the model in the 32-bit types for numeric stability, the model will have a lower step time and train equally as well in terms of the evaluation metrics such as accuracy, for a deeper understanding of mixed precision training, check out the tensorflow guide for [mixed precision:](https://www.tensorflow.org/guide/mixed_precision)

In [ ]:
# Turn on mixed precision training
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16') #Set global data to mixed precision

In [ ]:
mixed_precision.global_policy()

### Data Augmentation



In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

In [ ]:
datagen = ImageDataGenerator(zoom_range =0.4,horizontal_flip=True,shear_range =0.3)

# Load an image
img_sample = train_data.take(1)

for img,label in img_sample:
  img,label


#  Add the image to a batch.
img = tf.cast(tf.expand_dims(img, 0), tf.float32)
# iterator
aug_iter = datagen.flow(img, batch_size=1)

# generate samples and plot
fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(15,15))

# generate batch of images
for i in range(3):

	# convert to unsigned integers
	image = next(aug_iter)[0].astype('uint8')

	# plot image
	ax[i].imshow(image)
	ax[i].axis('off')



## Build feature extraction model

Building a feature extraction model for food classification simplifies complex data, improves model performance, and reduces training time by distilling raw information (like images) into meaningful features. It enables the model to capture critical patterns such as color, texture, and shape, which are essential for distinguishing between different food types. This process not only enhances classification accuracy but also helps handle variations in food images (e.g., lighting or background). Additionally, feature extraction allows for transfer learning, leveraging pre-trained models to accelerate training and optimize performance, ultimately creating a more efficient and robust classification system.

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import preprocessing

In [ ]:
# Finding our best base model to proceed with fine-tuning

input_shape = (224,224,3)
base_model = tf.keras.applications.EfficientNetB0(include_top = False)
base_model.trainable = False

# Creating a Functional API model
inputs = layers.Input(shape=input_shape,name = "input_layer")
# Since the Efficient models have rescaling built-in we will not include a layer for that
# x = preprocessing.Rescaling(1/255.)(x)

x = base_model(inputs,training=False) # Just to enforce no updating the model weights
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(len(class_names))(x)

# To make sure the output tensors are in float 32 for numerical stability
outputs = layers.Activation('softmax',dtype=tf.float32,name='Softmax_layer')(x)
model = tf.keras.Model(inputs,outputs)

# Compile model
model.compile(loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'],
              optimizer= tf.keras.optimizers.Adam(learning_rate= 0.001))

In [ ]:
model.summary()

In [ ]:
#check the dtype_pocies attribute of layers in our model
for layer in model.layers:
  print(layer.name,layer.trainable ,layer.dtype,layer.dtype_policy)

In [ ]:
# Check if GPU is present
!nvidia-smi -L

### Fit the baseline model

In this section, we will train a base model for food classification with the following configurations:

1. 3 epochs of training.
2. Use the ModelCheckpoint callback to save the best model weights during training.
3. Integrate Weights & Biases (W&B) for experiment tracking.

In [ ]:
# Install and prepare  wandb metrics
import wandb

from wandb.integration.keras import WandbMetricsLogger

In [ ]:
# configs for the weights and biases
configs =dict(
    batch_size =32,
    num_classes =len(class_names),
    shuffle_buffer = 1000,
    image_size = 224,
    image_channels = 3,
    earlystopping_patience =3,
    learning_rate = 1e-3,
    epochs = 3 # to be changed for the different models
)



In [ ]:
run =wandb.init(
    project = 'Food-Image-Classification',
    config =configs
)

# Using the exact replica of the Transfer learning data
Big_vision_history =model.fit(
    train_data,
    steps_per_epoch = int((0.5*len(train_data))), # 10% data
    epochs = configs['epochs'],
    validation_data = test_data.repeat(),
    validation_steps= int(0.15*len(test_data)), # 15 % of the data
    callbacks =[WandbMetricsLogger(log_freq =10)]
)
run.finish()


### Data Augmentation

Is the process of transforming images to create new ones, for training machine learning models

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

In [ ]:
datagen = ImageDataGenerator(zoom_range =0.2,horizontal_flip=True)

# Load an image
img_sample = train_data.take(1)

for img,label in img_sample:
  img,label


# iterator
aug_iter = datagen.flow(img, batch_size=32)

# generate samples and plot
fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(15,15))

# generate batch of images
for i in range(3):

	# convert to unsigned integers
	image = next(aug_iter)[0].astype('uint8')

	# plot image
	ax[i].imshow(image)
	ax[i].axis('off')



In [ ]:
aug_iter

In [ ]:
# Add Data Augmentation on the data as a layer (has benefits while using GPU)
tf.random.set_seed(42)
IMG_SIZE = (224,224)
data_augmentation = tf.keras.Sequential([
    layers.Input(shape = IMG_SIZE +(3,)),
    layers.RandomFlip('horizontal'),
    layers.RandomZoom(0.2),
    # layers.RandomRotation(0.2),
    # layers.RandomHeight(0.2),
    # layers.RandomWidth(0.2)
],name = 'data_augmentation')

In [ ]:
# Finding our best base model to proceed with fine-tuning
tf.random.set_seed(42)

input_shape = (224,224,3)
base_model = tf.keras.applications.EfficientNetB0(include_top = False)
base_model.trainable = False

# Creating a Functional API model
inputs = layers.Input(shape=input_shape,name = "input_layer")


x = data_augmentation(inputs)
x = base_model(inputs,training=False) # Just to enforce no updating the model weights
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(len(class_names))(x)

# To make sure the output tensors are in float 32 for numerical stability
outputs = layers.Activation('softmax',dtype=tf.float32,name='Softmax_layer')(x)
model2 = tf.keras.Model(inputs,outputs)

# Compile model
model2.compile(loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'],
              optimizer= tf.keras.optimizers.Adam(learning_rate= 0.001))

In [ ]:
# Update the epochs
configs['epochs'] = 6

run =wandb.init(
    project = 'Food-Image-Classification',
    config =configs
)
tf.random.set_seed(42)


# Using the exact replica of the Transfer learning data
Big_vision_history =model2.fit(
    train_data,
    steps_per_epoch = int((0.5*len(train_data))), # 50% data
    epochs = configs['epochs'],
    validation_data = test_data.repeat(),
    validation_steps= int(0.15*len(test_data)), # 15 % of the data
    callbacks =[WandbMetricsLogger(log_freq =10)]
)
run.finish()


In [ ]:
model2.evaluate(test_data)

In [ ]:
# Visualize the augmented Images
data_augmented=tf.keras.Sequential([
    layers.RandomFlip('horizontal'),
    layers.RandomZoom(0.2)
])



In [ ]:
Another_sample=train_data.take(1)

for image,label in Another_sample:
  image,label


In [ ]:
new_image =image[1]
# Add the image to a batch.
image = tf.cast(tf.expand_dims(new_image, 0), tf.float32)

In [ ]:
imag = image[0].numpy()
print(f"Data Type: {imag.dtype}, Min: {imag.min()}, Max: {imag.max()}")

In [ ]:
plt.figure(figsize=(10, 10))
for i in range(9):
  augmented_image = data_augmented(image)
  ax = plt.subplot(3, 3, i + 1)
  # Convert to numpy and check the value range
  img = augmented_image[0].numpy()  # Convert from Tensor to NumPy array

  if img.dtype == 'float32' or img.dtype == 'float64':  # Normalize if necessary
        img = img.clip(0, 1)  # Ensure values are within [0,1] if float

  plt.imshow(img)
  plt.axis("off")